In [8]:
import pandas as pd
from plotnine import *
from plotnine.data import *
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, precision_recall_curve
import numpy as np

df = pd.read_csv('data/cliqueData/politics.csv')
df['remainsValidFull'] = df['remainsValidPercentage']==1.0
#filter out singletons:
df = df[df['cliqueSize']>1]
print(df.dtypes)
print(df)
print(df.describe())


ComponentID                 int64
Method                     object
cliqueID                    int64
cliqueSize                  int64
remainsValidPercentage    float64
score                     float64
remainsValidFull             bool
dtype: object
      ComponentID  Method  cliqueID  cliqueSize  remainsValidPercentage  \
1           10893  greedy     11571           3                     1.0   
7           10893  greedy     11575           3                     1.0   
8            5598  greedy      5598           2                     1.0   
9            5598  greedy      5679           4                     1.0   
13           5598  greedy      5629           6                     1.0   
...           ...     ...       ...         ...                     ...   
1528         1018  greedy     37298           2                     1.0   
1544        13159  greedy     26640           2                     1.0   
1545        13159  greedy     22032           2                     1.0  